In [1]:
from lmnav.models.perceiver import Perceiver
import torch
import numpy as np

[2023-09-10 13:26:00,146] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


## Perceiver Set up

In [2]:
perceiver = Perceiver(
    input_channels = 768,          # number of channels for each token of the input
    input_axis = 1,              # number of axis for input data (2 for images, 3 for video)
    num_freq_bands = 64,          # number of freq bands, with original value (2 * K + 1)
    max_freq = 16384.,              # maximum frequency, hyperparameter depending on how fine the data is
    depth = 2,                   # depth of net. The shape of the final attention mechanism will be:
                                 #   depth * (cross attention -> self_per_cross_attn * self attention)
    num_latents = 256,           # number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim = 768,            # latent dimension
    cross_heads = 1,             # number of heads for cross attention. paper said 1
    latent_heads = 8,            # number of heads for latent self attention, 8
    cross_dim_head = 128,         # number of dimensions per cross attention head
    latent_dim_head = 128,        # number of dimensions per latent self attention head
    final_classifier_head = False,          # output number of classes
    attn_dropout = 0.1,
    ff_dropout = 0.1,
    weight_tie_layers = False,   # whether to weight tie layers (optional, as indicated in the diagram)
    fourier_encode_data = True,  # whether to auto-fourier encode the data, using the input_axis given. defaults to True, but can be turned off if you are fourier encoding the data yourself
    self_per_cross_attn = 7      # number of self attention blocks per cross attention
)
perceiver = perceiver.to('cuda')

## LLAMA Forward with Perceiver

In [3]:
from collections import namedtuple

from lmnav.common.config import Config
from lmnav.common.registry import registry

from lmnav.models import *
from lmnav.processors import *
from lmnav.common.episode_processor import apply_transforms_inputs

import torch
import einops


In [4]:
def _init_components(cfg_path, device):
    Args = namedtuple("Args", "cfg_path, model_type, gpu_id, options")
    args = Args(cfg_path, "llama_v2", 0, [])

    cfg = Config(args)

    model_config = cfg.model_cfg
    model_cls = registry.get_model_class(model_config.arch)
    model = model_cls.from_config(model_config).to(device)
    
    model.train()

    vis_processor_cfg = cfg.config.preprocess.vis_processor.train
    vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)

    return model, vis_processor


def test_construct_inputs(B, T):
    goals = torch.rand(B, 1, 3, 480, 640)
    rgbs = torch.rand(B, T, 3, 480, 640)
    actions = torch.randint(0, 4, (B, T))

    return goals, rgbs, actions

In [5]:
cfg_path = "/srv/flash1/pputta7/projects/lm-nav/exp_configs/video_nav_llama_train.yaml"
device = 'cuda'
B, T = 2, 20

model, vis_processor = _init_components(cfg_path, device)
model = model.half()

Loading VIT
Loading VIT Done
Loading Q-Former


Using pad_token, but it is not set yet.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Perceiver Mask Testing

In [8]:
x = torch.rand(100, 3, 224, 224).to('cuda').half()
embds = model.visual_encoder(x)
qtk = model.query_tokens.expand(embds.shape[0], -1, -1)
out = model.Qformer.bert(query_embeds = qtk,
                         encoder_hidden_states=embds,
                         return_dict=True)

In [11]:
out.last_hidden_state.shape

torch.Size([100, 32, 768])

In [13]:
from einops import repeat, rearrange
from torch import einsum

In [53]:
B, H, D = 1, 1, 512
Nkv = 5
Nq = 3
q = torch.rand(B, Nq, D)
k, v = (torch.rand(B, Nkv, D) for _ in range(2))
q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=H), (q, k, v))
sim = einsum('b i d, b j d -> b i j', q, k)

In [54]:
sim.shape

torch.Size([1, 3, 5])

In [56]:
mask = torch.ones(B, Nkv)
mask[..., -1] = 0
mask = rearrange(mask, 'b ... -> b (...)')
neg = -torch.finfo(sim.dtype).max
mask = repeat(mask, 'b j -> (b h) () j', h=H)
mask = mask.to(torch.bool)
mask

tensor([[[ True,  True,  True,  True, False]]])

In [57]:
sim.masked_fill_(~mask, neg)

tensor([[[ 1.3009e+02,  1.2752e+02,  1.2684e+02,  1.3022e+02, -3.4028e+38],
         [ 1.2227e+02,  1.1984e+02,  1.2209e+02,  1.2447e+02, -3.4028e+38],
         [ 1.3540e+02,  1.2980e+02,  1.2632e+02,  1.2921e+02, -3.4028e+38]]])

### Prompt Curation

In [6]:
prompt = "You are a navigational agent tasked with exploring an indoor environment to find a goal image. \
                  You can choose to move { left, right, forward, stop } at every step. The goal image is {}. \
                  You are provided with a video of your state action history: {}. Choose the next best action: {}"
 

In [83]:
from einops import rearrange, repeat
import importlib

B, T = 2, 10

rgbs = torch.rand(B, 3, T, 224, 224)
goals = torch.rand(B, 3, 1, 224, 224)
actions = [['left'], ['forward']]

rgbs_embd = model.embed_visual(rgbs)
goals_embd = model.embed_visual(goals)
rgbs_embd = rearrange(rgbs_embd, '(b t) ... -> b t ...', b=B)
goals_embd = rearrange(goals_embd, '(b t) ... -> b t ...', b=B)

In [84]:
mask = torch.ones(B, T + 1).bool().to('cuda')
mask = repeat(mask, 'b t -> b (t n)', n=32)

# construct the perceiver sequence: [goal] + [states]
video = torch.cat([goals_embd, rgbs_embd], dim=1)
video = rearrange(video, 'b t n h -> b (t n) h').half()
vid_embds = model.vidformer(video, mask=mask)
vid_embds = model.llama_proj(vid_embds)

vid_embds.shape

torch.Size([2, 256, 4096])

In [85]:
prompt_segs = prompt.split("{}")
prompt_tkns = [model.llama_tokenizer(seg, return_tensors='pt', add_special_tokens=(i == 0)) for i, seg in enumerate(prompt_segs) if len(seg)]
prompt_embd = [model.llama_model.get_input_embeddings()(tkns.input_ids.to(model.device)) for tkns in prompt_tkns]
prompt_embd = [embd.repeat(B, 1, 1) for embd in prompt_embd]

In [86]:
model.llama_tokenizer.batch_decode([e['input_ids'][0] for e in prompt_tkns])

['<s>You are a navigational agent tasked with exploring an indoor environment to find a goal image.                   You can choose to move { left, right, forward, stop } at every step. The goal image is ',
 '.                   You are provided with a video of your state action history: ',
 '. Choose the next best action: ']

In [87]:
action_tkns_t = model.tokenize_actions(actions)
actions_embd = model.embed_actions(action_tkns_t)

In [89]:
goals_embd = model.llama_proj(goals_embd.half())
goals_embd = goals_embd.squeeze(1)

In [90]:
embds = [prompt_embd[0], goals_embd, prompt_embd[1], vid_embds, prompt_embd[2], actions_embd] 

[torch.Size([2, 47, 4096]),
 torch.Size([2, 32, 4096]),
 torch.Size([2, 16, 4096]),
 torch.Size([2, 256, 4096]),
 torch.Size([2, 9, 4096]),
 torch.Size([2, 1, 4096])]